In [1]:
from connections import get_mongo_client

In [2]:
client = get_mongo_client()
db = client['task']
collection = db['task']

In [3]:
doc_max_num = collection.find({'data.katm_077.return.data.claims_wo_contracts.claim_wo_contract': {'$exists': True}},{'number': 1}).sort('number', -1).limit(1).next()

In [4]:
doc_max_num

{'_id': ObjectId('678f45df06a91d9394207cfa'), 'number': 8665600}